### Реализовать Out-of-Bag оценку(оценка на объектах из начальной выборки, не используемых при построении дерева) для каждого из деревьев леса

In [397]:
import matplotlib.pyplot as plt
import random

from sklearn.metrics import accuracy_score
from sklearn import datasets
from sklearn import model_selection

import numpy as np

In [398]:
random.seed(42)
 
def get_bootstrap(labels, N):
    return np.random.randint(0, len(labels), size=(N, len(labels)))

def get_oof(bootstrap_idx: np.array) -> np.array:
    oof_idx =[
        list(set(range(len(bootstrap_idx[0]))) - set(bootstrap))
        for bootstrap in bootstrap_idx
    ]
    return np.array(oof_idx)

In [399]:
def get_subsample(len_sample):
    # будем сохранять не сами признаки, а их индексы
    sample_indexes = [i for i in range(len_sample)]
    
    len_subsample = int(np.sqrt(len_sample))
    subsample = []
    
    random.shuffle(sample_indexes)
    for _ in range(len_subsample):
        subsample.append(sample_indexes.pop())
        
    return subsample

In [400]:
# Расчет критерия Джини

def gini(labels: np.array)-> float:
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [401]:
# Расчет качества

def quality(left_labels, right_labels, current_gini):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [402]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [403]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 1

    current_gini = gini(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    # выбор индекса из подвыборки длиной sqrt(n_features)
    subsample = get_subsample(n_features)
    
    for index in subsample:
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [404]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [405]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction        

In [406]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels):

    quality, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels)
    false_branch = build_tree(false_data, false_labels)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [407]:
def random_forest(data, labels, n_trees):
    forest, scores = [], []
    bootstrap_idx = get_bootstrap(data, N=n_trees)
    oof = get_oof(bootstrap_idx)
    
    for idx in bootstrap_idx:
        b_data, b_labels = data[idx], labels[idx]
        forest.append(build_tree(b_data, b_labels))
        
    for idx in oof:
        oof_data, oof_labels = data[idx], labels[idx]
        for tree in forest:
            y_pred = predict(oof_data, tree)
            score = accuracy_score(oof_labels, y_pred)
        scores.append(score)
        
    return forest, scores

In [408]:
# Функция классификации отдельного объекта

def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [409]:
# функция формирования предсказания по выборке на одном дереве

def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [410]:
# предсказание голосованием деревьев

def tree_vote(forest, data):

    # добавим предсказания всех деревьев в список
    predictions = []
    for tree in forest:
        predictions.append(predict(data, tree))
    
    # сформируем список с предсказаниями для каждого объекта
    predictions_per_object = list(zip(*predictions))
    
    # выберем в качестве итогового предсказания для каждого объекта то,
    # за которое проголосовало большинство деревьев
    voted_predictions = []
    for obj in predictions_per_object:
        voted_predictions.append(max(set(obj), key=obj.count))
        
    return voted_predictions


def evaluate_model(model, metric: callable, *args):
    if args:
        eval_sets = [(args[i], args[i+1]) for i in range(0, len(args), 2)]
        
        for x, y, in eval_sets:
            y_pred = tree_vote(model, x)
            score = metric(y, y_pred)
            print(score)
            

In [411]:
get_bootstrap(train_labels, N=1000)[0].shape[0] == train_labels.shape[0]

True

In [412]:
models = {}

for n_trees in [1, 3, 5, 10, 50, 100, 200]:
    models[n_trees] = random_forest(train_data, train_labels, n_trees)
    print(f"n_trees = {n_trees}, OOF-score = {round(np.mean(models[n_trees][1]), 4)}")
    evaluate_model(models[n_trees][0], accuracy_score, train_data, train_labels, test_data, test_labels)


n_trees = 1, OOF-score = 0.8134
0.9285714285714286
0.8933333333333333
n_trees = 3, OOF-score = 0.9074
0.9657142857142857
0.9466666666666667
n_trees = 5, OOF-score = 0.9326
0.9828571428571429
0.94
n_trees = 10, OOF-score = 0.9447
0.9971428571428571
0.9533333333333334
n_trees = 50, OOF-score = 0.9352
1.0
0.96
n_trees = 100, OOF-score = 0.9554
1.0
0.9733333333333334
n_trees = 200, OOF-score = 0.9735
1.0
0.9733333333333334


In [414]:
models, scores = random_forest(train_data, train_labels, 10)
np.mean(scores)

0.9680852561404661